# 카카오 API를 이용하여 검색한 키워드에 해당하는 값을 모두 출력합니다.

In [2]:
import requests
import folium
import collections

# 테스트 코드 작성

In [36]:
app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
params = {
    'query': '부어치킨',
    'page': 1,
    'rect': '',
}
headers  = {
    'Authorization': app_key
}
resp = requests.get(url, params=params, headers=headers)
# resp.json()

# 함수 정의

In [30]:
def get_store_list(start_x,start_y,end_x,end_y):
    offset = 0.01
    print(start_x,start_y,end_x,end_y)
    cnt = 1
    resp_list = []
    while True:
        jump_x = 0.5
        jump_y = 0.5
        params = {
            'query': '부어치킨',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        resp = requests.get(url, params=params, headers=headers)
        search_count = resp.json()['meta']['total_count']
#         print(search_count)
        if search_count > 45:
#             print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list        
            else:
#                 print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])
                
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)
        
def get_avg_point(x_list, y_list):
    x = sum(x_list) / len(x_list)
    y = sum(y_list) / len(y_list)
    return x, y

def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
#     disabled_sup = list(set(df_noman['장애인편의발급기형태']))
#     for x, y in addr_point_list:
#         if df_noman['장애인편의발급기형태'][cnt] == disabled_sup[0]:
#             add_marker(map_noman, x, y)
    return map_noman

def draw_marker(x, y, popup):
    marker = folium.Marker((x, y), popup=popup)
    popup = folium.Popup(marker, max_width=650)
    icon=folium.Icon(color='orange',icon='glyphicon glyphicon-cutlery')
    icon.add_to(marker)
    return marker

# 실제 구현

In [31]:
app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5
store_list = []
search_keyword = input('검색할 가게를 입력하세요')
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        store_list_one = get_store_list(start_x,start_y,end_x,end_y)
        store_list.extend(store_list_one)
        start_y = end_y
    start_x = end_x
# store_list # test

126 33 126.5 33.5
2
126 33.5 126.5 34.0
0
126 34.0 126.5 34.5
0
126 34.5 126.5 35.0
0
126 35.0 126.5 35.5
0
126 35.5 126.5 36.0
0
126 36.0 126.5 36.5
0
126 36.5 126.5 37.0
2
126 37.0 126.5 37.5
0
126 37.5 126.5 38.0
0
126 38.0 126.5 38.5
0
126 38.5 126.5 39.0
0
126.5 33 127.0 33.5
2
126.5 33.5 127.0 34.0
1
126.5 34.0 127.0 34.5
0
126.5 34.5 127.0 35.0
0
126.5 35.0 127.0 35.5
6
126.5 35.5 127.0 36.0
1
126.5 36.0 127.0 36.5
3
126.5 36.5 127.0 37.0
2
126.5 37.0 127.0 37.5
72
big data dividing...
126.5 37.0 126.75 37.25
1
126.75 37.0 127.0 37.25
3
126.5 37.25 126.75 37.5
31
paging...
31
paging...
31
126.75 37.25 127.0 37.5
40
paging...
40
paging...
40
126.5 37.5 127.0 38.0
75
big data dividing...
126.5 37.5 126.75 37.75
34
paging...
34
paging...
34
126.75 37.5 127.0 37.75
45
paging...
45
paging...
45
126.5 37.75 126.75 38.0
0
126.75 37.75 127.0 38.0
4
126.5 38.0 127.0 38.5
1
126.5 38.5 127.0 39.0
0
127.0 33 127.5 33.5
0
127.0 33.5 127.5 34.0
0
127.0 34.0 127.5 34.5
0
127.0 34.5 127.5 35.0


[{'address_name': '제주특별자치도 제주시 외도일동 532-3',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 부어치킨',
  'distance': '',
  'id': '17982927',
  'phone': '064-743-9917',
  'place_name': '부어치킨 외도점',
  'place_url': 'http://place.map.kakao.com/17982927',
  'road_address_name': '제주특별자치도 제주시 우정로11길 17',
  'x': '126.432805788736',
  'y': '33.4879000366299'},
 {'address_name': '제주특별자치도 서귀포시 대정읍 하모리 849-15',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 부어치킨',
  'distance': '',
  'id': '21462837',
  'phone': '064-792-8292',
  'place_name': '부어치킨 모슬포점',
  'place_url': 'http://place.map.kakao.com/21462837',
  'road_address_name': '제주특별자치도 서귀포시 대정읍 신영로 113',
  'x': '126.254231724832',
  'y': '33.2222168706101'},
 {'address_name': '충남 서산시 지곡면 무장리 920',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 부어치킨',
  'distance': '',
  'id': '19031670',
  'phone': '041-662

In [32]:
store_list2 = []
store_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in store_list)))
for i in store_list:
    j = i['place_name']
    tel = i['phone']
    addr = i['address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    store_list2.append({'jijum':j, 'tel':tel, 'addr':addr, 'x':x_list, 'y':y_list})
len(store_list2)

392

In [33]:
x_list = list(map(lambda i:i['x'], store_list2))
y_list = list(map(lambda i:i['y'], store_list2))
tel_list = list(map(lambda i:i['tel'], store_list2))
addr_list = list(map(lambda i:i['addr'], store_list2))
hosik2_map = folium.Map(location=get_avg_point(x_list, y_list),zoom_start=7)
for x, y, t, a in zip(x_list, y_list, tel_list,addr_list):
    marker = draw_marker(x, y, (t,a))
    marker.add_to(hosik2_map)
hosik2_map    

In [35]:
hosik2_map.save('hosik2.html')